# Horror Movie Data Analysis
### Author: Łukasz Ozimek
In this project I'll be taking a closer look at data regarding horror movies. I'll answer following questions:
1. [What was the number of movies released each month of each year, and the total number of movies were released each year?](#question_1)
2. [What was the average horror movie rating, budget and worldwide gross each month, each year, and what's the total average and median?](#question_2)
3. [Did horror movies get shorter?](#question_3)
4. [Did the amount of jumpscares in movies increase over the years?](#question_4)
4. [Did the quality of jumpscares rise?](#question_4.5)
5. [Is there a correlation between the IMDB rating and worldwide gross?](#question_5)
6. [Is there a correlation between the number of jumpscares and IMDB rating?](#question_6)
7. [Does the jumpscare rating affect the IMDB rating, and if so, does accounting for it in conjuction with the jumpscare count allow for better correlation with IMDB rating than jumpscare count alone?](#question_7)
8. [Did the public's acceptance of jumpscares decrease over the years?](#question_8)
9. [Does the movie's duration and the ratio of jumpscare/duration affect the IMDB rating? Which one is better correlated with the ratings?](#question_9)
10. [Does MPAA rating affect the IMDB rating and worldwide gross?](#question_10)
11. [What is the list of best rated movies and movies with highest gross values? What about directors?](#question_11)
12. [Do the movies that are on Netflix have a higher average IMDB rating and worldwide gross than the movies that aren't?](#question_12)



Tools: Python (Jupyter Notebook), Tableau Public (data visualization)

Since the dataset is a single table there is no need to use SQL or other DBMS.

Link to the movie dataset used in this project: https://www.sfu.ca/~nimtiaz/horrormovies_imdb.csv

In [37]:
# Module imports
import pandas as pd
import numpy as np

First I'll import the data and do some exploratory data analysis.

In [38]:
df = pd.read_csv('./horrormovies_imdb.csv')
df.head()

,Movie Name,Director,Year,Jump Count,Jump Scare Rating,Imdb,MPAA Rating,Genre,Month,Budget,Worldwide Gross,Netflix (US),Duration,Rotten Tomatoes,Language,Country,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,10 Cloverfield Lane,Dan Trachtenberg,2016,8,2.5,7.2,PG-13,"Drama, Horror, Mystery",March,"$15,000,000","$110,216,998",No,104,NaN,English,USA,NaN,NaN,NaN,NaN
1,28 Days Later,Danny Boyle,2003,10,3.0,7.6,R,"Drama, Horror, Sci-Fi",June,"$8,000,000","$85,720,385",No,113,NaN,English,UK,NaN,NaN,NaN,NaN
2,28 Weeks Later,Juan Carlos Fresnadillo,2007,17,3.5,7.0,R,"Drama, Horror, Sci-Fi",May,"$15,000,000","$64,238,440",No,100,NaN,English,UK,NaN,NaN,NaN,NaN
3,30 Days of Night,David Slade,2007,9,2.5,6.6,R,"Action, Horror, Thriller",October,"$30,000,000","$75,505,977",No,113,NaN,English,USA,NaN,NaN,NaN,NaN
4,31,Rob Zombie,2016,8,2.5,5.1,R,"Horror, Thriller",September,"$1,500,000","$850,419",No,102,NaN,English,USA,NaN,NaN,NaN,NaN


In [39]:
# Creating a short loop that creates a simple report on distinct value counts and nulls for each column
for col in df.columns:
    print('Column name: ', col)
    print('\t Count of distinct values: ', len(df[col].unique()))
    print('\t Has null: ', df[col].isnull().values.any())
    print()

Column name:  Movie Name
	 Count of distinct values:  530
	 Has null:  False

Column name:  Director
	 Count of distinct values:  394
	 Has null:  False

Column name:  Year
	 Count of distinct values:  54
	 Has null:  False

Column name:  Jump Count
	 Count of distinct values:  30
	 Has null:  False

Column name:  Jump Scare  Rating
	 Count of distinct values:  11
	 Has null:  False

Column name:  Imdb
	 Count of distinct values:  49
	 Has null:  False

Column name:  MPAA Rating
	 Count of distinct values:  8
	 Has null:  False

Column name:  Genre
	 Count of distinct values:  77
	 Has null:  False

Column name:  Month
	 Count of distinct values:  12
	 Has null:  False

Column name:  Budget
	 Count of distinct values:  144
	 Has null:  True

Column name:  Worldwide Gross
	 Count of distinct values:  488
	 Has null:  True

Column name:  Netflix  (US)
	 Count of distinct values:  2
	 Has null:  False

Column name:  Duration
	 Count of distinct values:  74
	 Has null:  False

Column name:

The preliminary check has revealed a few things:
* The columns that reveal null values are: Budget, Worldwide Gross, Country, Rotten Tomatoes, Language and all unnamed
* The Rotten Tomatoes column and all Unnamed columns consist only of null values (except Unnamed 19, but since there is no way on knowing what this feature is it should be removed)
* The number of distinct jumpsacre count and movie duration values are surprisingly low, but there is no way to suspect it's an error in the data

First I'll remove the columns filled with null values, then I'll check value counts for all columns with null values to find out whether they can be dropped or not.

In [40]:
df.drop(columns=['Rotten Tomatoes', 'Unnamed: 16', 'Unnamed: 17' ,'Unnamed: 18', 'Unnamed: 19'], inplace=True)
df.head()

,Movie Name,Director,Year,Jump Count,Jump Scare Rating,Imdb,MPAA Rating,Genre,Month,Budget,Worldwide Gross,Netflix (US),Duration,Language,Country
0,10 Cloverfield Lane,Dan Trachtenberg,2016,8,2.5,7.2,PG-13,"Drama, Horror, Mystery",March,"$15,000,000","$110,216,998",No,104,English,USA
1,28 Days Later,Danny Boyle,2003,10,3.0,7.6,R,"Drama, Horror, Sci-Fi",June,"$8,000,000","$85,720,385",No,113,English,UK
2,28 Weeks Later,Juan Carlos Fresnadillo,2007,17,3.5,7.0,R,"Drama, Horror, Sci-Fi",May,"$15,000,000","$64,238,440",No,100,English,UK
3,30 Days of Night,David Slade,2007,9,2.5,6.6,R,"Action, Horror, Thriller",October,"$30,000,000","$75,505,977",No,113,English,USA
4,31,Rob Zombie,2016,8,2.5,5.1,R,"Horror, Thriller",September,"$1,500,000","$850,419",No,102,English,USA


In [41]:
# Loop to count values in selected columns
for col in ['Budget', 'Worldwide Gross', 'Country', 'Language']:
    print('Column %s has %s null values.' % (col, df[col].isna().sum()))

Column Budget has 112 null values.
Column Worldwide Gross has 56 null values.
Column Country has 519 null values.
Column Language has 520 null values.


In [42]:
df.drop(columns=['Country', 'Language'], inplace = True)

Budget, Country and Language features have a large number of null values. Since this project focuses on analysis there is no need to remove them, but it's important to keep that knowledge in mind. In this example one could also notice the importance of thorough exploratory data analysis, since the first few rows of the dataset visible above have non-null Country and Language values. The Country and Language features will be removed anyway, since they convey little information and won't be used in this project.

The EDA can be considered done, but before we get into the analysis I'll preprocess the data. A few questions on my list require additional metrics to be created:
* Question 9 requires a metric that links jumpscare count and IMDB rating. I'll create it by multiplying the IMDB rating times the jumpscare count, thus allowing to check if movies with high amount of jumpscares used to get better ratings
* Question 10 requires a metric that link duration and jumpscare count. I'll divide the jumpscare count by duration thus allowing to measure number of jumpscares per minute

Let's start with creating these metrics.

In [43]:
# Creating a copy of the dataset to modify it
df_mod = df.copy()

# Converting monetary values from string to numeric

# Function to convert the values
def convert_monetary(list_col):
    for idx, item in enumerate(list_col):
        if type(item) is not float:
            list_col[idx] = int(item.replace('$','').replace(',',''))
            
budget_values = list(df_mod['Budget'])
gross_values = list(df_mod['Worldwide Gross'])

# Using the function
convert_monetary(budget_values)
convert_monetary(gross_values)

# Saving to dataframe
df_mod['Budget'] = budget_values
df_mod['Worldwide Gross'] = gross_values

# Jumpscare count & IMDB rating metric:
df_mod['jump_IMDB_coef'] = df_mod['Jump Count'] * df_mod['Imdb']

# Jumpscare count & duration metric:
df_mod['jump_duration_ratio'] = df_mod['Jump Count'] / df_mod['Duration']

df_mod.head()

,Movie Name,Director,Year,Jump Count,Jump Scare Rating,Imdb,MPAA Rating,Genre,Month,Budget,Worldwide Gross,Netflix (US),Duration,jump_IMDB_coef,jump_duration_ratio
0,10 Cloverfield Lane,Dan Trachtenberg,2016,8,2.5,7.2,PG-13,"Drama, Horror, Mystery",March,15000000.0,110216998.0,No,104,57.6,0.076923
1,28 Days Later,Danny Boyle,2003,10,3.0,7.6,R,"Drama, Horror, Sci-Fi",June,8000000.0,85720385.0,No,113,76.0,0.088496
2,28 Weeks Later,Juan Carlos Fresnadillo,2007,17,3.5,7.0,R,"Drama, Horror, Sci-Fi",May,15000000.0,64238440.0,No,100,119.0,0.170000
3,30 Days of Night,David Slade,2007,9,2.5,6.6,R,"Action, Horror, Thriller",October,30000000.0,75505977.0,No,113,59.4,0.079646
4,31,Rob Zombie,2016,8,2.5,5.1,R,"Horror, Thriller",September,1500000.0,850419.0,No,102,40.8,0.078431


In [44]:
# Before the Dataframe is saved I need to remove spaces from the column names as it causes Tableau to read the data incorrectly
cols = list(df_mod.columns)
for idx, col in enumerate(cols):
    cols[idx] = col.replace(' ','_')
df_to_save = df_mod.copy()
dictionary = dict(zip(list(df_mod.columns), cols))
df_to_save.rename(columns=dictionary, inplace=True)

# Saving the new csv file to use in Tableau
df_to_save.to_csv('./horror_mod.csv',index=False,sep=';')

Now it's time to dive into the analysis!

<a id='question_1'></a>
#### Question 1
**What was the number of movies released each month of each year, and the total number of movies were released each year?**

**Fig 1.** Fragment of the graph showing number of movies released each month of each of selected years
![Question 1.1](imgs/img1.png)

**Fig 2.** Graph showing number of movies released each year
![Question 1.2](imgs/img2.png)

**Fig 3.** Graph showing number of movies released each month
![Question 1.3](imgs/img3.png)

Since visualizing movies released each month of each year yields a large graph no matter how it's done I've decided to create a bar chart (for its ease of reading) and apply filters. It's impossible to show it in Jupyter Notebook in a reasonable amount of space, so feel free to look at it in the Tableau file in this repo. If there were fewer years in the dataset it could be possible to use color to visualize months and years without filters, but with the current dataset such representation would be very difficult to read.

In fig. 2 we can see that as expected the number of horror movies released each year kept growing, but it's worth noting, that since 2017 fewer horror movies are being released each year, and the number of horror movies released in 2019 is as low as in 2010.

In fig. 3 we can see, that October (most likely because of Halloween) is the month, when most horror movies are released, while in December the number of horror movies released is lowest.

<a id='question_2'></a>
#### Question 2
**What was the average horror movie rating, budget and worldwide gross each month, each year, and what's the total average and median?**


**Fig 4.** Fragments of graphs visualizing IMDB rating, budget and worldwide gross each month of each year.
![Question 2](imgs/img4.jpg)

I've created the graphs once again using bar charts and filters. This time I decided to color the bars by month, because the graph is much easier to read that way.

I calculated the total averages and medians in Python with below code:

In [45]:
for i in ['Budget','Worldwide Gross']:
    total_average = df_mod[i].mean()
    total_median = df_mod[i].median()
    print('Total average %s: %.2f; Total median: %s' % (i, total_average, total_median))

Total average Budget: 19837730.04; Total median: 10000000.0
Total average Worldwide Gross: 79255233.97; Total median: 36782199.0


<a id='question_3'></a>
#### Question 3
**Did horror movies get shorter?**

**Fig 5.** Graph visualising horror movie duration by year.
![Question 3](imgs/img5.png)

As we can see in the graph in fig 5, horror movies tended to have similar duration times through the years. However, there was a noticeable decrease in the years 2008 - 2015, but since 2015 the average movie duration started rising again back to its pre-2008 value.

<a id='question_4'></a>
#### Question 4
**Did the amount of jumpscares in movies increase over the years?**

**Fig 6.** Graph visualizing the average jumpscare count in movies over the years
![Question 4](imgs/img6.png)

In the graph we can see that the average jumpscare count indeed did increase over the years, however the increase is much smaller than expected. It can be seen, that jumpscares became popular in the 1980's, and then it was gradually increasing, except a few years when the average was much lower. The lowest bars for each decade also achieve higher values after the year 2000 than they did before, which also suggests the increasing popularity of jumpscares.


<a id='question_4.5'></a>
#### Question 5
**Did the quality of jumpscares rise?**

**Fig 7.** Graph visualizing jumpscare scare rating by year.
![Question 5](imgs/img7.png)

The jumpscare scare rating seems to increase over the years.

When analyzing figures 6 and 7 it's also important to keep in mind the number of movies made each years. The jumpscare count and rating seem to be highest in the 80's, but one should keep in mind that only a few horror movies were released in those years, so the higher average values are most likely caused by the fact that the few movies released contained jumpscares, while in the 2000's only some of many horror movies released did so. I also recommend looking at the Tableau project file in this repo, and changing from average to sum or median. You'll see that the median outcome is similar to the average, thus we can conclude that the outliers had little effect on the mean. Looking at sum values you'll notice that my observation regarding number of movies released each year and it's effect on jumpscare count and ratings was correct.


<a id='question_5'></a>
#### Question 6
**Is there a correlation between the IMDB rating and worldwide gross?**

**Fig 8.** Graph visualizing worldwide gross by IMDB rating.
![Question 6](imgs/img8.png)

Surprisingly there seems to be no strong correlation between movie's IMDB rating and it's worldwide gross. However it can be seen in the graph, that the lowest worldwide gross values for movies with high IMDB rating is higher than for movies with low IMDB rating, so if a company wants to be sure to make a profit they should take care to create a movie that will achieve higher IMDB rating, since doing so will ensure that the lowest possible gross will be higher than in the case of low rated movie.


<a id='question_6'></a>
#### Question 7
**Is there a correlation between the number of jumpscares and IMDB rating?**

**Fig 9.** Graph visualising IMDB rating by number of jumpscares.
![Question 7](imgs/img9.png)

There seems to be an obvious correlation between number of jumpascares and IMDB rating, with the exception of movies where number of jumpscares is between 21-28, but other than that it seems that the more jumpscares a movie has, the lower its IMDB rating.


<a id='question_7'></a>
#### Question 8
**Does the jumpscare rating affect the IMDB rating, and if so, does accounting for it in conjuction with the jumpscare count allow for better correlation with IMDB rating than jumpscare count alone?**

In order to answer the question once could add average scare rating in Tableau as either size or color to the graph from the previous question. I'll add it as size, and budget as color, to find out whether a high budget (marketing etc.) is what affects the ratings of movies with 21-28. 

**Fig 10.** Graph visualising IMDB rating by jumpscare count, scare rating and budget.
![Question 8.1](imgs/img10.png)

Quite surprisingly, the IMDB ratings seem to get lower even when scare ratings grow, which means that even when a movie has many good jumpscares, the audience still responds negatively to their large count. We can also see that the budget doesn't seem to be the deciding factor why movies with 22-28 jumpscares seem to break the decreasing trend. It means that even horror movies with large budgets can fail if they have too many jumpscares. Therefore the question regarding why movies with 21-28 jumpscares requires break the trend requires more analysis, and probably more data (for example number of IMDB reviews etc.)

The correlations will be calculated with below code (I've decided to use Pearson's correlation coefficient, as it's a good middle ground between Spearman's and Kendall's correlation coefficients):

In [46]:
from scipy.stats import pearsonr

print('Jumpscare count and IMDB ratings')
r, p = pearsonr(df_mod['Jump Count'],df_mod['Imdb'])
print("\t Pearson’s correlation coefficient: %.3f, Two-tailed p-value: %.2f"%(r,p))

print('Jumpscare rating and IMDB ratings')
r, p = pearsonr(df_mod['Jump Scare  Rating'],df_mod['Imdb'])
print("\t Pearson’s correlation coefficient: %.3f, Two-tailed p-value: %.2f"%(r,p))

# I couldn't find a function to calculate correlation from multiple variables, so I made my own
def multiple_correlation(var_x, var_y, var_z):
    # Calculates correlation of var_z from both var_x and var_y
    # Formula taken from: https://www.real-statistics.com/correlation/multiple-correlation/
    r_xz, p_xz = pearsonr(var_x,var_z)
    r_yz, p_yz = pearsonr(var_y,var_z)
    r_xy, p_xy = pearsonr(var_x,var_y)
    R_z_xy = np.sqrt((r_xz**2 + r_yz**2 - (2*r_xz*r_xy*r_yz))/(1-(r_xy**2)))   
    return R_z_xy

R_z_xy = multiple_correlation(df_mod['Jump Count'],df_mod['Jump Scare  Rating'],df_mod['Imdb'])
print()
print('Multiple correlation: %f' % (R_z_xy))

Jumpscare count and IMDB ratings
	 Pearson’s correlation coefficient: -0.282, Two-tailed p-value: 0.00
Jumpscare rating and IMDB ratings
	 Pearson’s correlation coefficient: -0.277, Two-tailed p-value: 0.00

Multiple correlation: 0.285833


As we can see in the code output above the pairing of jumpscare count and jumpscare scare rating is better correlated to IMDB ratings than just jumpscare count, but the correlation is still quite small.

<a id='question_8'></a>
#### Question 9
**Did the public's acceptance of jumpscares decrease over the years?**

Since this project is all about practicing my Tableau and analysis skills I'll answer this question in two ways. First I'll plot IMDB rating against years with coloring by average jumpscare count (figure 11). The second way is to use the metric I created earlier (figure 12).

**Fig 11.** Graph visualising jumpscare popularity over the years.
![Question 9.1](imgs/img11.png)

Horror movies seem to be getting lower and lower average IMDB ratings as their release year increases, and we can see that jumpscare count might have something to do with that, as in later years there are much more dark dots showing higher average jumpscare count. We can observe, that untill the 1990's the jumpscares were getting increasingly popular, but then their popularity began to fade. We can also see that since 2015 the trend seems to change and the ratings grow despite high jumpscare count.

**Fig 12.** Graph visualising jumpscare popularity over the years.
![Question 9.1](imgs/img12.png)

Using a numeric and continuous year values shows clearly a large gap between 1940 and 1960, when the horror movie industry seems to have really taken off. Since the calculated metric seems to have high variance over the years I've decided to add a polynomial (4th order) trend line to ease the analysis. We can see, that the trend line supports the conclusions made from figure 11.


<a id='question_9'></a>
#### Question 10
**Does the movie's duration and the ratio of jumpscare/duration affect the IMDB rating? Which one is better correlated with the ratings?**

**Fig 13.** Graph visualising duration and jumpscare per minute by IMDB ratings. The red line is average duration, while the blue area is average jumpscare per minute.
![Question 10](imgs/img13.png)

The graph in figure 13 shows, that longer horror movies on average tend to get higher IMDB ratings, and as expected at some point (0.1268 jumpscares pre minute, which is 7,6 jumpscares per hour) of the jumpscare/duration ratio the IMDB ratings begin to take a dive, and decrease rapidly, further supporting the thesis that movie directors should use jumpscares with moderation.

Correlation calculation:

In [47]:
print('Duration and IMDB ratings')
r, p = pearsonr(df_mod['Duration'],df_mod['Imdb'])
print("\t Pearson’s correlation coefficient: %.3f, Two-tailed p-value: %.2f"%(r,p))

print('Jumpscare per minute and IMDB ratings')
r, p = pearsonr(df_mod['jump_duration_ratio'],df_mod['Imdb'])
print("\t Pearson’s correlation coefficient: %.3f, Two-tailed p-value: %.2f"%(r,p))

Duration and IMDB ratings
	 Pearson’s correlation coefficient: 0.399, Two-tailed p-value: 0.00
Jumpscare per minute and IMDB ratings
	 Pearson’s correlation coefficient: -0.331, Two-tailed p-value: 0.00


As we can see above the movie's duration is better correlated with IMDB ratings than the ratio, and even batter than the jumpscare count.

<a id='question_10'></a>
#### Question 11
**Does MPAA rating affect the IMDB rating and worldwide gross?**

**Fig 14.** Graph visualising IMDB rating and Worldwide gross by MPAA rating
![Question 11](imgs/img14.png)

As we can see in the graph the MPAA rating has little effect on the ratings, altough PG rated movies seem to have the highest ratings of all. I've also created bands with standard error values. The MPAA rating definietely does have an effect on worldwide gross. The standard deviation is large for most categories, which is caused by successful hit movies in those categories. But it can be easily observed, that the PG and PG-13 movies achive much higher gross values than any other categories.


<a id='question_11'></a>
#### Question 12
**What is the list of best rated movies and movies with highest gross values? What about directors?**

In order to answer that question I'll use Python and Pandas

In [48]:
print('10 movies with highest ratings:')
display(df_mod[['Movie Name','Director','Imdb']].nlargest(10,'Imdb'))
print('----------------------------------------------------------------------------------------------')
print('10 movies with highest worldwide gross:')
display(df_mod[['Movie Name','Director','Worldwide Gross']].nlargest(10,'Worldwide Gross'))
print('----------------------------------------------------------------------------------------------')
# Since Steven Spielberg is on the list twice we only have top 9 directors, the table below removes one of Spielberg's 
#      movies and takes an additional director to create the top 10 directors list
print('10 movies with highest worldwide gross:')
display(df_mod[['Movie Name','Director','Worldwide Gross']].nlargest(11,'Worldwide Gross').drop(202))

10 movies with highest ratings:


,Movie Name,Director,Imdb
311,Seven,David Fincher,8.6
470,The Silence of the Lambs,Jonathan Demme,8.6
280,Psycho,Alfred Hitchcock,8.5
18,Alien,Ridley Scott,8.4
22,Aliens,James Cameron,8.4
469,The Shining,Stanley Kubrick,8.4
259,Pan's Labyrinth,Guillermo del Toro,8.2
209,Jurassic Park,Steven Spielberg,8.1
314,Shutter Island,Martin Scorsese,8.1
471,The Sixth Sense,M. Night Shyamalan,8.1


----------------------------------------------------------------------------------------------
10 movies with highest worldwide gross:


,Movie Name,Director,Worldwide Gross
210,Jurassic World,Colin Trevorrow,1.670401e+09
211,Jurassic World: Fallen Kingdom,J.A. Bayona,1.308468e+09
209,Jurassic Park,Steven Spielberg,1.030314e+09
197,It,Andy Muschietti,7.004493e+08
471,The Sixth Sense,M. Night Shyamalan,6.728063e+08
179,I Am Legend,Francis Lawrence,5.853490e+08
538,World War Z,Marc Forster,5.400079e+08
432,The Meg,Jon Turteltaub,5.302437e+08
202,Jaws,Steven Spielberg,4.712030e+08
198,It Chapter Two,Andy Muschietti,4.592457e+08


----------------------------------------------------------------------------------------------
10 movies with highest worldwide gross:


,Movie Name,Director,Worldwide Gross
210,Jurassic World,Colin Trevorrow,1.670401e+09
211,Jurassic World: Fallen Kingdom,J.A. Bayona,1.308468e+09
209,Jurassic Park,Steven Spielberg,1.030314e+09
197,It,Andy Muschietti,7.004493e+08
471,The Sixth Sense,M. Night Shyamalan,6.728063e+08
179,I Am Legend,Francis Lawrence,5.853490e+08
538,World War Z,Marc Forster,5.400079e+08
432,The Meg,Jon Turteltaub,5.302437e+08
198,It Chapter Two,Andy Muschietti,4.592457e+08
389,The Exorcist,William Friedkin,4.413061e+08


<a id='question_12'></a>
#### Question 13
**Does MPAA rating affect the IMDB rating and worldwide gross?**

**Fig 14.** Graph visualising IMDB rating and Worldwide gross by being on Netflix in the US. The gross is shown with bars, while the rating is shown with a Gant bar and it's value label.
![Question 13](imgs/img15.png)

In the figure 14 we can observe, that both ratings and gross values (especially the latter) are much higher for movies that aren't on Netflix in the US. The difference in ratings is most likely caused by the fact, that many older movies that got better ratings than newer movies, are not in Netflix. The difference of worldwide grosses would require more in-depth analysis.